# Übung 2 - Aufgabe 1 - Klassifikation von Katzen und Hunden

In dieser Aufgabe soll der im Buch vorgestellte Datensatz verwendet werden, um Katzen und Hunde zu klassifizieren. Im Gegensatz zur vorherigen Aufgabe soll in dieser allerdings mittels fortgeschritteneren Netzen klassifiziert werden. Verwenden Sie dazu CNNs folgender Typen:

- Ein eigenes CNN angelehnt an das aus dem Buch
- Ein VGG Ihrer Wahl
- Ein ResNet Ihrer Wahl

Verwenden Sie zusätzlich ein Netz auf Basis von Vision Transformern. Vergleichen Sie die Ergebnisse. Gehen Sie auf diese Weise vor ist es sinnvoll sich für das Training eine wiederverwendbare Funktion zu schreiben.

Diese Aufgabe ist folgendermaßen aufgebaut:

1. Vornehmen nötiger Imports
2. Laden der Daten
3. Definition des eigenen CNN
4. Anlegen der neuronalen Netze
5. Training und Tests der neuronalen Netze

## 1. Nötige Imports vornehmen

Die folgende Code-Zelle importiert die notwendigen Bibliotheken, die im Folgenden benötigt werden.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import timm


## 2. Trainingsdaten laden

Zunächst werden die Trainingsdaten aus dem Verzeichnis geladen. Hierbei können Sie wie im Buch beschrieben vorgehen.

In [ ]:
path = 'Data/'

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_set = ImageFolder(root=path + 'train', transform=transform)
valid_set = ImageFolder(root=path + 'valid', transform=transform)


Im Folgenden kann eines der Bilder angezeigt werden.

In [ ]:
def imshow(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)

imshow(train_set[30][0])

Die Daten werden nun einem Data Loader übergeben, welcher eine Zufallsverteilung und eine Organisation in "batches" vornimmt.


In [ ]:
train_loader = DataLoader(train_set,shuffle=True,batch_size=64,num_workers=0)
valid_loader = DataLoader(valid_set,batch_size=64,num_workers=0)

## 3. Definition des eigenen CNN

Nutzen Sie die vorgegebene Klasse CNN, um ein eigenes CNN zu implementieren. Aufgrund der Größe der Eingabebilder, sollte dieses mindestens 4-5 Conv2D-Schichten enthalten.

In [ ]:
num_classes = 2

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128*16*16, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


## 4. Anlegen der neuronalen Netze

Legen Sie in der nächsten Zelle die vier Netze der vorgegebenen Typen an. Denken Sie daran die Ausgabe auf die Anzahl der vorliegenden Klassen anzupassen. Beim VGG lässt sich dies im letzten Layer der Sequenz "classifier" und beim Vision Transformer im letzten Layer der Sequenz "heads" setzen. Das ResNet können Sie wie im Kapitel 3 beschrieben anpassen.

In [ ]:
def prepare_vgg(model, n_classes):
    model.classifier[6] = nn.Linear(4096, n_classes)
    return model

def prepare_resnet(model, n_classes):
    model.fc = nn.Linear(model.fc.in_features, n_classes)
    return model

def prepare_vit(model, n_classes):
    # Angenommen, der Vision Transformer hat einen "head" oder "classifier" am Ende
    # Sie müssen dies entsprechend dem von Ihnen verwendeten Modell anpassen.
    model.head = nn.Linear(model.head.in_features, n_classes)
    return model

cnn = CNN()
vgg = prepare_vgg(models.vgg16(pretrained=True), num_classes)
resnet = prepare_resnet(models.resnet18(pretrained=True), num_classes)

# Für den Vision Transformer:
# Sie müssen ein Modell initialisieren, bevor Sie es verwenden können. 
# Hier habe ich es als `vit_model` bezeichnet. 
# Sie können eine Bibliothek wie `timm` verwenden, um ein vorab trainiertes Vision Transformer Modell zu erhalten.
vit_model = timm.create_model('vit_base_patch16_224', pretrained=True)
vit = prepare_vit(vit_model, num_classes)


## 5. Training des Modells

Schreiben Sie in die Funktion "train_model" die Trainingslogik der neuronalen Netze. Hier können Sie ähnlich wie bei der MNIST-Aufgabe vorgehen. Die Funktion "plot_results" kann Genauigkeit und Verlust als Plot darstellen.

In [ ]:
def plot_results(epochs, training_acc, testing_acc, training_loss, testing_loss):
    plt.plot(range(epochs), training_acc, label="train_acc")
    plt.plot(range(epochs), testing_acc, label="valid_acc")
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.show()

    plt.plot(range(epochs), training_loss, label="train_loss")
    plt.plot(range(epochs), testing_loss, label="valid_loss")
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.show()

def train_model(model, epochs=10, learning_rate=0.001):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    training_loss = []
    testing_loss = []
    training_acc = []
    testing_acc = []

    if torch.cuda.is_available():
        model = model.to("cuda")

    with tqdm(range(epochs)) as iterator:
        for epoch in iterator:
            train_loss = 0
            train_acc = 0

            model.train()
            for images, labels in train_loader:
                if torch.cuda.is_available():
                    images = images.to("cuda")
                    labels = labels.to("cuda")

                optimizer.zero_grad()
                outputs = model(images)
                loss = loss_fn(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                _, predicted = outputs.max(1)
                train_acc += (predicted == labels).sum().item()

            training_acc.append(train_acc/len(train_set))
            training_loss.append(train_loss/len(train_loader))

            test_loss = 0
            test_acc = 0
            model.eval()
            with torch.no_grad():
                for images, labels in valid_loader:
                    if torch.cuda.is_available():
                        images = images.to("cuda")
                        labels = labels.to("cuda")

                    outputs = model(images)
                    loss = loss_fn(outputs, labels)

                    test_loss += loss.item()
                    _, predicted = outputs.max(1)
                    test_acc += (predicted == labels).sum().item()

                testing_acc.append(test_acc/len(valid_set))
                testing_loss.append(test_loss/len(valid_loader))

            iterator.set_postfix_str(f"train_acc: {train_acc/len(train_set):.2f} test_acc: {test_acc/len(valid_set):.2f} train_loss: {train_loss/len(train_loader):.2f} test_loss: {test_loss/len(valid_loader):.2f}")

    plot_results(epochs, training_acc, testing_acc, training_loss, testing_loss)



Wenn Sie Ihre Funktion "train_model" korrekt implementiert haben und Ihre Modelle definiert sind, können Sie ihre Modelle in den folgenden vier Zellen trainieren lassen. Da der Datensatz recht groß ist, würde ich ein Training auf 3-10 Epochen begrenzen. Falls Sie eine Cuda-fähige Grafikkarten besitzen, können Sie auch länger trainieren.

In [ ]:
train_model(cnn)

In [ ]:
train_model(vgg)

In [ ]:
train_model(resnet)

In [ ]:
train_model(vit)